In [ ]:
#dataフォルダからファイルを取得
import pandas as pd
import glob
datalists = glob.glob('data/*.xlsx')
print(datalists)

In [ ]:
#出力されたファイル一覧から、今回使用するファイルをリスト内のインデックス番号でjに指定する。例えば['AAA.xlsx','BBB.xlsx','CCCCxlsx']が
#表示されたときAAAファイルは0（ゼロ）、BBBファイルは１、CCCファイルは２
j=2
#選択したファイルをDataframeに読み込み
input_file = pd.ExcelFile(datalists[j])
selected_sheet_df = input_file.parse(0)
x=int()
name = ''
print(selected_sheet_df)
def new_list(x):
#必要な情報（DNA以外）を取り出し①
    s1 = selected_sheet_df.loc[:,('Order','Family','Genus','Species','目','科','属','種')]
#必要なDNA配列数を取り出します。取り出した後は百分率に変換②
    s_d = selected_sheet_df.iloc[:,x]
    l=[]
    s=s_d.sum()
    for i in s_d:
        m=int(i) / int(s)
        l.append(m)
    k = pd.DataFrame(l).rename(columns={0:selected_sheet_df.columns[x]})
#①と②のデータを結合
    df = pd.merge(k, s1,left_index=True, right_index=True)
# 検体名と個々のデータを行方向で紐づけ
    name = df.columns[0]
    df['検体名'] = name
    df = df.rename(columns={name:'DNA比率'})
    df = df.fillna('Others')
    return df.iloc[:,[9,0,1,2,3,4,5,6,7,8]]

In [3]:
#夫々のサンプルデータを行方向に連結
n=[]
for y in range(len(selected_sheet_df.columns)-10):
    n.append(y+1)
df1 = pd.DataFrame(index=[], columns=[])
for x in n:
    df1 = pd.concat([df1,new_list(x)])

In [4]:
#DNAデータがゼロのものを削除、また同じ生物種のデータをグループ化
df1 = df1[['検体名', 'DNA比率', 'Family', 'Genus', 'Species', '科', '属', '種']].groupby(['検体名', 'Family', 'Genus', 'Species', '科', '属', '種'],
       as_index=False ).sum()
df1 = df1.iloc[:,[0,7,1,2,3,4,5,6]]
df_N_new = df1[df1['DNA比率'] >= 0.0000001]

In [5]:
#蜜源分類をマージ。その後蜜源分類の空白を「3 その他」に
df_蜜源分類 = pd.read_excel("蜜源分類.xlsx")
df_N_new_蜜源分類 = pd.merge(df_N_new, df_蜜源分類, on='属', how='left')
df_N_new_蜜源分類 = df_N_new_蜜源分類.fillna({'蜜源分類': '3 その他'})

In [6]:
#環境省外来種ファイルを読み込み
input_file_name2 = "fuka_plant2.xlsx"
input_book2 = pd.ExcelFile(input_file_name2)
input_sheet_name2 = input_book2.sheet_names
#余計な行を削除
input_sheet_df2 = input_book2.parse(input_sheet_name2[0], skiprows=4)
#余計な行を更に削除。学名がNAの行は除外
input_sheet_df2 = input_sheet_df2.dropna(subset=['学名'], axis=0)
#必要な列を抽出
input_sheet_df2 = input_sheet_df2[['No.','カテゴリ','学名','和名（別名、流通名）']]
#データをマージするために目印となるカラム名を変更
input_sheet_df2 = input_sheet_df2.rename(columns={'学名':'Species'})
#外来種データをマージ。NAは===で表示
df_input_sheet_蜜源分類_外来種 = pd.merge(df_N_new_蜜源分類, input_sheet_df2, on='Species', how='left')
df_input_sheet_蜜源分類_外来種 = df_input_sheet_蜜源分類_外来種.drop(columns=['No.','和名（別名、流通名）'], axis=1)

In [ ]:
#環境省希少種ファイルを読み込み
input_file_name3 = "redlist2020_ikansoku.xlsx"
df_K = pd.ExcelFile(input_file_name3)
input_sheet_name3 = df_K.sheet_names
input_sheet_df3 = df_K.parse(input_sheet_name3[0])
#データをマージするために目印となるカラム名を学名からSpecies変更
input_sheet_df3 = input_sheet_df3.rename(columns={'学名':'Species'})
#希少種データをマージ
df_input_sheet_蜜源分類_外来種_希少種 = pd.merge(df_input_sheet_蜜源分類_外来種, input_sheet_df3, on='Species', how='left')
df_input_sheet_蜜源分類_外来種_希少種 = df_input_sheet_蜜源分類_外来種_希少種.drop(columns=['分類群','和名'], axis=1)
df_input_sheet_蜜源分類_外来種_希少種 = df_input_sheet_蜜源分類_外来種_希少種.fillna('-')
df = df_input_sheet_蜜源分類_外来種_希少種
df = df.replace('-','')
df['外来種/希少種'] = df['カテゴリ'].str.cat(df['カテゴリー'])
df.drop(columns=['カテゴリ', 'カテゴリー'], inplace=True)
df

In [8]:
#前処理が終了したエクセルファイルを出力
name = datalists[j]
df.to_excel(name,index=False)